In [2]:
import sqlite3
import pandas as pd
from notebook_tools import side_by_side, get_sql_table_info

# import plotly.express as px
# import seaborn as sns

connection = sqlite3.connect("db/vivino.db")
cursor = connection.cursor()

get_sql_table_info(cursor, save_to_file=True, print_out=True)



Table: countries
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	code                    VARCHAR      1       None      1    
	name                    VARCHAR      0       None      0    
	regions_count           INTEGER      0       None      0    
	users_count             INTEGER      0       None      0    
	wines_count             INTEGER      0       None      0    
	wineries_count          INTEGER      0       None      0    

Table: grapes
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	id                      INTEGER      1       None      1    
	name                    VARCHAR      0       None      0    

Table: wineries
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY
	id                      INTEGER      1       None      1    
	name                    VARCHAR      0       None      0    

Table: flavor_groups
Columns:
	NAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY

'\n\nTable: countries\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tcode                    VARCHAR      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\tregions_count           INTEGER      0       None      0    \n\tusers_count             INTEGER      0       None      0    \n\twines_count             INTEGER      0       None      0    \n\twineries_count          INTEGER      0       None      0    \n\nTable: grapes\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tid                      INTEGER      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\nTable: wineries\nColumns:\n\tNAME                    TYPE         NOTNULL DEFAULT   PRIMARY KEY\n\tid                      INTEGER      1       None      1    \n\tname                    VARCHAR      0       None      0    \n\nTable: flavor_groups\nColumns:\n\tNAME                    T

### 1. Ratings overview: 10 most profitable wines?

In [69]:
q = """
SELECT
    wines.ratings_average AS 'avg rating',
    wines.ratings_count AS 'rating count'
FROM
    wines
"""

wine_df = pd.read_sql_query(q, connection)

q = """
SELECT
    vintages.ratings_average AS 'avg rating',
    vintages.ratings_count AS 'rating count'
FROM
    vintages
WHERE
    ratings_count AND ratings_average > 0
"""

vintages_df = pd.read_sql_query(q, connection)

side_by_side({
    "WINES Ratings": wine_df.describe(),
    # "WINES Correlation": wine_df.corr(),
    "VINTAGES Ratings": vintages_df.describe(),
    # "VINTAGES Correlation": vintages_df.corr(),
})

print("\nYeah these ratings are not that useful...")
print("Let's combine the rating and count into one 'Popularity' score.")
print("Let's then multiply by the price to get a final pofitabilty score.")

,avg rating,rating count
count,1020.000000,1020.000000
mean,4.443431,6387.334314
std,0.118239,14022.610539
min,4.100000,27.000000
25%,4.400000,474.000000
50%,4.400000,1697.500000
75%,4.500000,5644.750000
max,4.800000,157944.000000
,avg rating,rating count
count,1540.000000,1540.000000



Yeah these ratings are not that useful...
Let's combine the rating and count into one 'Popularity' score.
Let's then multiply by the price to get a final pofitabilty score.


In [94]:
q = """
WITH leaderboard AS (
    SELECT
        wines.name AS Wine,
        CAST(AVG(vintages.price_euros) AS INT) AS Price,
        AVG(wines.ratings_average) AS Rating,
        SUM(wines.ratings_count) AS Count,
        wines.ratings_average * LOG(wines.ratings_count * 0.0001 - 3) AS Popularity
    FROM
        vintages
        JOIN wines ON wines.id = vintages.wine_id
    GROUP BY
        wines.id
    )
SELECT
    Wine,
    Price AS 'AVG Vintage Price',
    Rating,
    Count,
    Popularity,
    Price * Popularity AS Score
FROM
    leaderboard
ORDER BY
    Score DESC
LIMIT 10
"""

print("Considering wines only:")
pd.read_sql_query(q, connection).head(60)

Considering wines only:


,Wine,AVG Vintage Price,Rating,Count,Popularity,Score
0,Sassicaia,990,4.6,2691150,4.094548,4053.602325
1,Pauillac (Premier Grand Cru Classé),1098,4.6,736150,2.942325,3230.672887
2,Opus One,720,4.6,462318,3.093902,2227.609198
3,Brut Champagne,437,4.6,731885,4.902989,2142.606160
4,Tignanello,430,4.5,2280208,4.730412,2034.077234
5,Château Margaux (Premier Grand Cru Classé),1135,4.6,634900,0.856099,971.671851
6,Cabernet Sauvignon,177,4.6,157944,5.092292,901.335622
7,Pauillac (Grand Cru Classé),415,4.4,167424,1.811719,751.863443
8,Sauternes,973,4.7,397134,0.705090,686.052812
9,Costasera Amarone della Valpolicella Classico,195,4.3,186794,3.448895,672.534611


In [93]:
q = """
WITH leaderboard AS (
    SELECT
        vintages.name AS Vintage,
        CAST(AVG(vintages.price_euros) AS INT) AS Price,
        vintages.ratings_average AS Rating,
        SUM(vintages.ratings_count) AS Count,
        vintages.ratings_average * LOG(vintages.ratings_count * 0.001 - 3) AS Popularity
    FROM
        vintages
    GROUP BY
        vintages.id
    )
SELECT
    Vintage,
    Price AS 'Price €',
    Rating,
    Count,
    Popularity,
    Price * Popularity AS Score
FROM
    leaderboard
ORDER BY
    Score DESC
LIMIT 10
"""

print("Considering vintages only:")
pd.read_sql_query(q, connection).head(60)

Considering vintages only:


,Vintage,Price €,Rating,Count,Popularity,Score
0,Tenuta San Guido Sassicaia 2015,1237,4.6,6699,2.613188,3232.513462
1,Dom Pérignon Brut Champagne 2006,440,4.6,21121,5.787638,2546.560693
2,Louis Roederer Cristal Brut Champagne (Millési...,2071,4.7,4757,1.150427,2382.534895
3,Opus One Opus One 2015,1665,4.7,4948,1.361068,2266.178348
4,Dom Pérignon Brut Champagne 2004,460,4.6,14650,4.905099,2256.345658
5,Opus One Opus One 2013,768,4.6,7011,2.774962,2131.171004
6,Antinori Tignanello 2016,420,4.6,13709,4.736845,1989.474911
7,Tenuta San Guido Sassicaia 2011,661,4.6,7165,2.850229,1884.001386
8,Tenuta San Guido Sassicaia 2014,576,4.5,8182,3.215238,1851.977283
9,Tenuta San Guido Sassicaia 2013,632,4.6,7103,2.820267,1782.408730


### 2. Marketing budget: which country?

In [6]:
q = """
SELECT
    countries.name,
    countries.users_count,
    countries.users_count / wineries_count AS users_per_winery
FROM
    countries
ORDER BY countries.users_count DESC
LIMIT 10
"""

df = pd.read_sql_query(q, connection)
df

q = """
SELECT
    countries.name,
    countries.users_count,
    countries.users_count / wineries_count AS users_per_winery
FROM
    countries
ORDER BY users_per_winery DESC
LIMIT 10
"""

df2 = pd.read_sql_query(q, connection)
df2

side_by_side({
    "Ordered by user count": df,
    "Ordered by users/winery": df2
})

,name,users_count,users_per_winery
0,États-Unis,12273684,436
1,France,5973301,88
2,Italie,4270717,100
3,Allemagne,2549989,186
4,Espagne,2264396,125
5,Suisse,1601799,416
6,Portugal,1123535,192
7,Australie,1022965,73
8,Argentine,629532,117
9,Chili,326757,56


### 3. Three Relevant Wineries

In [180]:
q = """
WITH temp2 AS (
    WITH temp AS (
        SELECT
            vintages.name AS Vintage,
            wines.name AS Wine,
            vtr.rank AS Rank,
            vintages.ratings_average AS Ratings,
            vintages.ratings_count AS Count,
            vintages.year as Year
        FROM
            vintages
            JOIN wines ON vintages.wine_id = wines.id
            LEFT JOIN vintage_toplists_rankings AS vtr ON vtr.vintage_id = vintages.id
        WHERE
            vintages.year != 'N.V.'
            AND vintages.year > 2010
            AND Ratings != 0
    )
    SELECT
        SUBSTR(Vintage, 1, LENGTH(Vintage) - LENGTH(Wine) - 6) AS Winery,
        MAX(Year) AS LatestYear,
        MAX(Ratings) AS Ratings,
        SUM(Count) AS Count
    FROM temp
    GROUP BY
        Winery
    ORDER BY
        Count DESC
)
SELECT
    Winery,
    LatestYear AS 'Latest Year',
    Ratings AS 'Best Rating',
    Count,
    Ratings * LOG(Count * 0.000001 + 1.5) AS Score
FROM
    temp2
ORDER BY
    Score DESC
LIMIT 3
"""

df = pd.read_sql_query(q, connection)
side_by_side({
    "": df
})

,Winery,Latest Year,Best Rating,Count,Score
0,Antinori,2021,4.700000,75989,0.928500
1,Tenuta San Guido,2020,4.700000,48554,0.892654
2,Casanova di Neri,2018,4.800000,17436,0.869330


### 4. Coffee, Toast, Cream, Green Apple and Citrus

In [51]:
q = """
WITH concat_keywords AS (
    SELECT
        wines.name AS Wine,
        wines.ratings_average AS Rating,
        keywords_wine.count AS count,
        keywords.name AS keyword,
        GROUP_CONCAT(keywords.name) AS AllKeywords
    FROM
        keywords_wine
        JOIN wines ON keywords_wine.wine_id = wines.id
        JOIN keywords ON keywords_wine.keyword_id = keywords.id
    WHERE
        keyword IN ("coffee", "toast", "green apple", "cream", "citrus")
        AND count > 9
    GROUP BY
        wines.id
)
SELECT
    Wine,
    AllKeywords,
    Rating
FROM
    concat_keywords
WHERE
    AllKeywords LIKE '%coffee%'p
    AND AllKeywords LIKE '%toast%'
    AND AllKeywords LIKE '%green apple%'
    AND AllKeywords LIKE '%cream%'
    AND AllKeywords LIKE '%citrus%'
ORDER BY
    Rating DESC
"""

df = pd.read_sql_query(q, connection)
df.head(60)

,Wine,AllKeywords,Rating
0,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,"citrus,toast,coffee,coffee,cream,toast,green a...",4.7
1,Sauternes,"green apple,coffee,toast,coffee,citrus,cream,t...",4.7
2,P2 Plénitude Brut Champagne,"toast,coffee,coffee,cream,toast,green apple,ci...",4.7
3,Cristal Brut Champagne (Millésimé),"toast,coffee,green apple,citrus,coffee,cream,t...",4.6
4,Vintage,"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
5,Brut Champagne,"toast,coffee,cream,toast,coffee,green apple,ci...",4.6
6,R.D Extra Brut Champagne (Récemment Dégorgé),"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
7,Blanc des Millénaires,"toast,coffee,coffee,cream,toast,citrus,green a...",4.6
8,Sir Winston Churchill Brut Champagne,"toast,coffee,coffee,cream,toast,green apple,ci...",4.6
9,Grande Cuvée,"toast,coffee,coffee,cream,toast,citrus,green a...",4.6


### 5. TOP 5 Wines for the TOP 3 Grapes

In [181]:
q = """
WITH ranked_wines AS (
    SELECT
        wines.name AS Wine,
        grapes.name AS Grape,
        wines.ratings_average AS 'Rating',
        countries.name AS Country,
        ROW_NUMBER() OVER(PARTITION BY grapes.name ORDER BY wines.ratings_average DESC) AS wine_rank
    FROM
        wines
        JOIN regions ON wines.region_id = regions.id
        JOIN countries ON regions.country_code = countries.code
        JOIN most_used_grapes_per_country AS mugpc ON countries.code = mugpc.country_code
        JOIN grapes ON grapes.id = mugpc.grape_id
    WHERE
        grapes.name IN (
            SELECT
                distinct(name)
            FROM
                most_used_grapes_per_country AS mugpc
                JOIN grapes ON grapes.id = mugpc.grape_id
            ORDER BY
                wines_count DESC
            LIMIT 3
        )
    ORDER BY
        wines.ratings_average DESC
)
SELECT
    Wine,
    Grape,
    Country,
    Rating
FROM
    ranked_wines
WHERE
    wine_rank <= 5
ORDER BY
    Grape, Rating DESC;
"""

wines_df = pd.read_sql_query(q, connection)
wines_df.head(50)

,Wine,Grape,Country,Rating
0,Cabernet Sauvignon,Cabernet Sauvignon,États-Unis,4.8
1,Mágico,Cabernet Sauvignon,Espagne,4.8
2,IX Estate Red,Cabernet Sauvignon,États-Unis,4.7
3,Special Selection Cabernet Sauvignon,Cabernet Sauvignon,États-Unis,4.7
4,Unico Reserva Especial Edición,Cabernet Sauvignon,Espagne,4.7
5,Amarone della Valpolicella Classico Riserva,Chardonnay,Italie,4.8
6,Cabernet Sauvignon,Chardonnay,États-Unis,4.8
7,Fratini Bolgheri Superiore,Chardonnay,Italie,4.8
8,Cristal Rosé Vinothèque,Chardonnay,France,4.8
9,Batard-Montrachet Grand Cru,Chardonnay,France,4.8


### 6. AVG Wine rating for each country and each vintage

In [185]:
q = """
SELECT
    AVG(wines.ratings_average) AS AVGRating,
    AVG(wines.ratings_count) AS AVGCount,
    countries.name AS Country
FROM
    wines
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
WHERE
    wines.ratings_average AND wines.ratings_count > 0
GROUP BY
    Country
ORDER BY
    AVGRating DESC
"""

wine_df = pd.read_sql_query(q, connection)

q = """
SELECT
    AVG(vintages.ratings_average) AS AVGRating,
    AVG(vintages.ratings_count) AS AVGCount,
    countries.name AS Country
FROM
    vintages
    JOIN wines ON wines.id = vintages.wine_id
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
WHERE
    vintages.ratings_average AND vintages.ratings_count > 0
GROUP BY
    Country
ORDER BY
    AVGRating DESC
"""

vintages_df = pd.read_sql_query(q, connection)

side_by_side({
    "Wines": wine_df,
    "Vintages": vintages_df
})


,AVGRating,AVGCount,Country
0,4.500000,894.000000,Israël
1,4.500000,589.785714,Allemagne
2,4.490541,11273.824324,États-Unis
3,4.480000,2822.800000,Moldavie
4,4.472727,2778.545455,Hongrie
5,4.459091,2727.045455,Afrique du Sud
6,4.458333,4344.833333,Australie
7,4.447130,6419.362538,France
8,4.443617,7943.723404,Espagne
9,4.435714,2230.500000,Portugal


### 7. TOP 5 Cabernet Sauvignon pour Monsieur Vieille-Pie

In [3]:
q = """
SELECT
    wines.name AS Wine,
    grapes.name AS Grape,
    wines.ratings_average AS Rating,
    wines.ratings_count AS 'Ratings Count',
    wines.ratings_average * LOG(wines.ratings_count * 0.000001 + 2.2) AS Score
FROM
    wines
    JOIN regions ON regions.id = wines.region_id
    JOIN countries ON countries.code = regions.country_code
    JOIN most_used_grapes_per_country AS mugpc ON mugpc.country_code = countries.code
    JOIN grapes ON grapes.id = mugpc.grape_id
WHERE Grape = 'Cabernet Sauvignon'
ORDER BY
    Score DESC
LIMIT 5
"""

df = pd.read_sql_query(q, connection)
df

,Wine,Grape,Rating,Ratings Count,Score
0,Cabernet Sauvignon,Cabernet Sauvignon,4.6,157944,1.713654
1,Unico,Cabernet Sauvignon,4.7,45140,1.650844
2,Special Selection Cabernet Sauvignon,Cabernet Sauvignon,4.7,41236,1.647292
3,Cabernet Sauvignon,Cabernet Sauvignon,4.8,2941,1.646414
4,Opus One,Cabernet Sauvignon,4.6,77053,1.643916
